In [1]:
import numpy as np
import xarray as xr
import os
import intake

import getpass
import azure.storage.blob
import zarr

### Preliminaries

In [2]:
######################
# Azure blob storage
######################
# connection string (from azure web login, select your storage account, then "Access keys")
connection_string = getpass.getpass()

    
# format storage
container_client = azure.storage.blob.ContainerClient.from_connection_string(
    connection_string, container_name="mpctransfer")

 ········


In [3]:
###################
# Models
###################
from utils import gardsv_ssp_dict, gardsv_var_dict, deepsdbc_dict

In [17]:
#################
# Data access
#################

# Complete catalog
cat = intake.open_esm_datastore(
    "https://cpdataeuwest.blob.core.windows.net/cp-cmip/version1/catalogs/global-downscaled-cmip6.json"
)

# function to grab all variables and SSPs for singe model/method
def grab_model(method, model, scenarios):
    # Search catalogue for method, model, all SSPs, all vars
    dsets = cat.search(
        method=method,
        source_id=model,
        experiment_id=scenarios,
        timescale='day'
    ).to_dataset_dict()
    
    # Concat along SSP dimension and return
    ds_ssp = []
    for key in list(dsets.keys()):
        # Get single ssp file
        ds_tmp = dsets[key]
        
        # Add ssp dimension
        if 'experiment_id' in ds_tmp.attrs:
            ds_tmp = ds_tmp.assign_coords(ssp = ds_tmp.attrs['experiment_id'])
        else:
            ds_tmp = ds_tmp.assign_coords(ssp = ds_tmp.attrs['intake_esm_attrs:experiment_id'])
        
        # Drop member ID
        ds_tmp = ds_tmp.drop('member_id')
        
        # for some models/methods we are missing precip
        # so need to fill with NaNs
        if 'pr' not in ds_tmp.data_vars:
            ds_tmp['pr'] = xr.full_like(ds_tmp['tasmax'], np.NaN)
            ds_tmp['pr'].attrs = {'units':'NaN'}
        
        # Append
        ds_ssp.append(ds_tmp)
    
    # Rechunk for faster compuations
    ds_out = xr.concat(ds_ssp, dim='ssp')
    ds_out = ds_out.chunk({'ssp':1, 'lat':360, 'lon':360,
                               'time':tuple(ds_out.time.groupby(ds_out.time.dt.year).count().to_numpy())})
    return ds_out

In [10]:
##################
# Convert units
##################
def convert_units(ds):
    if 'tasmax' in ds.data_vars and ds.tasmax.attrs['units'] == 'K':
        ds['tasmax'] = ds['tasmax'] - 273.15
    if 'tasmin' in ds.data_vars and ds.tasmin.attrs['units'] == 'K':
        ds['tasmin'] = ds['tasmin'] - 273.15
    if 'pr' in ds.data_vars and ds.pr.attrs['units'] == 'kg m-2 s-1':
        ds['pr'] = ds['pr'] * 86400
    
    return ds

In [6]:
#########
# Dask
#########
import dask_gateway
gateway = dask_gateway.Gateway()

# cluster options
cluster_options = gateway.cluster_options()
cluster_options["worker_memory"] = 24
cluster_options["worker_cores"] = 1

# start cluster
cluster = gateway.new_cluster(cluster_options)
client = cluster.get_client()
cluster.scale(40)

# dashboard link
print(cluster.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.a9c5699985bf45698601e564e89aa9fb/status


# GARD-SV

In [7]:
# Preliminaries
method = 'GARD-SV'
models = list(gardsv_ssp_dict.keys())

assert models == list(gardsv_var_dict.keys())

## Simple metrics

### Annual avgs

In [33]:
# loop through models: RUNTIME IS AROUND 5 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, gardsv_ssp_dict[model])
    
    ds = convert_units(ds)
    ds['tas'] = (ds['tasmin'] + ds['tasmax']) / 2.
    
    # Annual averages
    ds_final = ds.resample(time='1Y').mean()
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/avg/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


BCC-CSM2-MR

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


CanESM5

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MIROC6

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MPI-ESM1-2-HR


### 1-day max

In [34]:
# loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, gardsv_ssp_dict[model])

    ds = convert_units(ds)
    ds['tas'] = (ds['tasmin'] + ds['tasmax']) / 2.
    
    # Annual maxs
    ds_final = ds.resample(time='1Y').max()
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/max/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


BCC-CSM2-MR

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


CanESM5

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MIROC6

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MPI-ESM1-2-HR


### 5-day max (pr)

In [ ]:
import xclim
xclim.set_options(cf_compliance="log");

In [ ]:
# loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, gardsv_ssp_dict[model])
    ds = convert_units(ds)
    
    if ds['pr'].attrs['units'] == 'NaN':
        print(model + ' contains no precip')
        continue
    
    # Select only precip
    ds = ds.drop(['tasmin', 'tasmax'])
    ds.pr.attrs['units'] = 'mm/day'
    
    # Compute
    ds_final = xclim.indicators.icclim.RX5day(ds=ds, freq='Y')
        
    # Storage options
    ds_final = xr.Dataset({'RX5day':ds_final})
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/max5d/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)

### Dry days

In [12]:
# Function for longest consecutive spell if needed
def n_longest_consecutive(ds):
    ds_out = ds.cumsum(dim='time') - ds.cumsum(dim='time').where(ds == 0).ffill(dim='time').fillna(0)
    return ds_out.max(dim='time')

In [17]:
# loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, gardsv_ssp_dict[model])
    ds = convert_units(ds)
    
    if ds['pr'].attrs['units'] == 'NaN':
        print(model + ' contains no precip')
        continue
    
    # Select only precip
    ds = ds.drop(['tasmin', 'tasmax'])
    
    # Compute
    # Number of dry days
    ds_tmp_0 = (ds == 0.).resample(time='1Y').sum() # 0mm
    ds_tmp_1 = (ds < 1.).resample(time='1Y').sum() # less than 1mm
    # Longest sonsecutive dry day streak
    ds_tmp_0c = (ds == 0.).resample(time='1Y').apply(n_longest_consecutive) # 0mm longest consecutive
    ds_tmp_1c = (ds < 1.).resample(time='1Y').apply(n_longest_consecutive) # less than 1mm longest consecutive
    # Merge
    ds_final = xr.merge([ds_tmp_0.rename({'pr':'count_eq_0'}),
                         ds_tmp_0c.rename({'pr':'streak_eq_0'}),
                         ds_tmp_1.rename({'pr':'count_lt_1'}),
                         ds_tmp_1c.rename({'pr':'streak_lt_1'})])
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/dry/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


BCC-CSM2-MR contains no precip

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


CanESM5

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MIROC6 contains no precip

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MPI-ESM1-2-HR


## Less simple metrics

In [11]:
# Function for longest consecutive spell if needed
def n_longest_consecutive(ds):
    ds_out = ds.cumsum(dim='time') - ds.cumsum(dim='time').where(ds == 0).ffill(dim='time').fillna(0)
    return ds_out.max(dim='time')

### Wet days

In [18]:
%%time
    
# Load quantiles
ds_q_era5 = xr.open_dataset('../data/quantiles/era5_precip_quantiles_gardsv.nc')
ds_q_gmfd = xr.open_dataset('../data/quantiles/gmfd_precip_quantiles_gardsv.nc')
    
# loop through models: RUNTIME IS AROUND 20 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, gardsv_ssp_dict[model])
    ds = convert_units(ds)
    
    if ds['pr'].attrs['units'] == 'NaN':
        print(model + ' contains no precip')
        continue
        
    # Select only precip
    ds = ds.drop(['tasmin', 'tasmax'])
    
    ## Calculate metrics
    var_id = 'pr'
    ds_tmp_out = []
    for rp in ['q99', 'rp10']:
        # Get above/below binary
        ds_tmp_q_era5 = ds[var_id] > ds_q_era5[var_id + '_' + rp]
        ds_tmp_q_gmfd = ds[var_id] > ds_q_gmfd[var_id + '_' + rp]
        
        # Count of hot days
        ds_tmp_q_era5_count = ds_tmp_q_era5.resample(time='1Y').sum()
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_count': ds_tmp_q_era5_count}))
        ds_tmp_q_gmfd_count = ds_tmp_q_gmfd.resample(time='1Y').sum()
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_count': ds_tmp_q_gmfd_count}))
        
        # Longest consecutive hot day streak
        ds_tmp_q_era5_streak = ds_tmp_q_era5.resample(time='1Y').apply(n_longest_consecutive)
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_streak': ds_tmp_q_era5_streak}))
        ds_tmp_q_gmfd_streak = ds_tmp_q_gmfd.resample(time='1Y').apply(n_longest_consecutive)
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_streak': ds_tmp_q_gmfd_streak}))
        
    # Merge metrics and append
    ds_final = xr.merge(ds_tmp_out)
        
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':20, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/wet/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')
    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


BCC-CSM2-MR contains no precip

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


CanESM5

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MIROC6 contains no precip

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MPI-ESM1-2-HR
CPU times: user 4min 24s, sys: 2.49 s, total: 4min 27s
Wall time: 29min 1s


### Hot days

In [20]:
%%time
    
# Load quantiles
ds_q_era5 = xr.open_dataset('../data/quantiles/era5_temperature_quantiles_gardsv.nc')
ds_q_gmfd = xr.open_dataset('../data/quantiles/gmfd_temperature_quantiles_gardsv.nc')
    
# loop through models: RUNTIME IS AROUND 10 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, gardsv_ssp_dict[model])
    ds = convert_units(ds)
    ds['tas'] = (ds['tasmax'] + ds['tasmin']) / 2.
    
    # Drop precip
    ds = ds.drop(['pr'])
    
    ## Calculate metrics
    ds_tmp_final = []
    for var_id in ['tasmin','tasmax','tas']:
        ds_tmp_out = []
        for rp in ['q99', 'rp10']:
            # Get above/below binary
            ds_tmp_q_era5 = ds[var_id] > ds_q_era5[var_id + '_' + rp]
            ds_tmp_q_gmfd = ds[var_id] > ds_q_gmfd[var_id + '_' + rp]
        
            # Count of hot days
            ds_tmp_q_era5_count = ds_tmp_q_era5.resample(time='1Y').sum()
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_count': ds_tmp_q_era5_count}))
            ds_tmp_q_gmfd_count = ds_tmp_q_gmfd.resample(time='1Y').sum()
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_count': ds_tmp_q_gmfd_count}))
        
            # Longest consecutive hot day streak
            ds_tmp_q_era5_streak = ds_tmp_q_era5.resample(time='1Y').apply(n_longest_consecutive)
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_streak': ds_tmp_q_era5_streak}))
            ds_tmp_q_gmfd_streak = ds_tmp_q_gmfd.resample(time='1Y').apply(n_longest_consecutive)
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_streak': ds_tmp_q_gmfd_streak}))
        
        # Merge RPs and append
        ds_out = xr.merge(ds_tmp_out)
        ds_tmp_final.append(ds_out)
    
    # Merge variables
    ds_final = xr.merge(ds_tmp_final)
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':20, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/hot/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')
    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


BCC-CSM2-MR

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


CanESM5

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MIROC6

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MPI-ESM1-2-HR
CPU times: user 42min 53s, sys: 12.1 s, total: 43min 6s
Wall time: 3h 14min 15s


# DeepSD-BC

In [21]:
# Preliminaries
method = 'DeepSD-BC'
models = list(deepsdbc_dict.keys())

## Simple metrics

### Annual avgs

In [ ]:
# loop through models: RUNTIME IS AROUND 20 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model,
                   [ssp for ssp in list(deepsdbc_dict[model].keys()) if 'pr' in deepsdbc_dict[model][ssp]])
    
    ds = convert_units(ds)
    ds['tas'] = (ds['tasmin'] + ds['tasmax']) / 2.
    
    # Annual averages
    ds_final = ds.resample(time='1Y').mean()
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/avg/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)

### 1-day maxs

In [ ]:
# loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model,
                   [ssp for ssp in list(deepsdbc_dict[model].keys()) if 'pr' in deepsdbc_dict[model][ssp]])

    ds = convert_units(ds)
    ds['tas'] = (ds['tasmin'] + ds['tasmax']) / 2.
    
    # Annual maxs
    ds_final = ds.resample(time='1Y').max()
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/max/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)

### 5-day max (pr)

In [21]:
import xclim
xclim.set_options(cf_compliance="log");

In [ ]:
# loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model,
                    [ssp for ssp in list(deepsdbc_dict[model].keys()) if 'pr' in deepsdbc_dict[model][ssp]])
    ds = convert_units(ds)
    
    # Select only precip
    ds = ds.drop(['tasmin', 'tasmax'])
    ds.pr.attrs['units'] = 'mm/day'
    
    # Compute
    ds_final = xclim.indicators.icclim.RX5day(ds=ds, freq='Y')
        
    # Storage options
    ds_final = xr.Dataset({'RX5day':ds_final})
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/max5d/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


### Dry days

In [17]:
# Function for longest consecutive spell if needed
def n_longest_consecutive(ds):
    ds_out = ds.cumsum(dim='time') - ds.cumsum(dim='time').where(ds == 0).ffill(dim='time').fillna(0)
    return ds_out.max(dim='time')

In [18]:
# loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model,
                    [ssp for ssp in list(deepsdbc_dict[model].keys()) if 'pr' in deepsdbc_dict[model][ssp]])
    ds = convert_units(ds)
    
    # Select only precip
    ds = ds.drop(['tasmin', 'tasmax'])
    
    # Compute
    # Number of dry days
    ds_tmp_0 = (ds == 0.).resample(time='1Y').sum() # 0mm
    ds_tmp_1 = (ds < 1.).resample(time='1Y').sum() # less than 1mm
    # Longest sonsecutive dry day streak
    ds_tmp_0c = (ds == 0.).resample(time='1Y').apply(n_longest_consecutive) # 0mm longest consecutive
    ds_tmp_1c = (ds < 1.).resample(time='1Y').apply(n_longest_consecutive) # less than 1mm longest consecutive
    # Merge
    ds_final = xr.merge([ds_tmp_0.rename({'pr':'count_eq_0'}),
                         ds_tmp_0c.rename({'pr':'streak_eq_0'}),
                         ds_tmp_1.rename({'pr':'count_lt_1'}),
                         ds_tmp_1c.rename({'pr':'streak_lt_1'})])
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':30, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/dry/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MRI-ESM2-0


## Less simple metrics

In [22]:
# Function for longest consecutive spell if needed
def n_longest_consecutive(ds):
    ds_out = ds.cumsum(dim='time') - ds.cumsum(dim='time').where(ds == 0).ffill(dim='time').fillna(0)
    return ds_out.max(dim='time')

### Wet days

In [23]:
%%time
    
# Load quantiles
ds_q_era5 = xr.open_dataset('../data/quantiles/era5_precip_quantiles_nex-cil-deepsd.nc')
ds_q_era5['lon'] = np.where(ds_q_era5['lon'] > 180, ds_q_era5['lon'] - 360, ds_q_era5['lon'])
ds_q_era5 = ds_q_era5.sortby('lon')

ds_q_gmfd = xr.open_dataset('../data/quantiles/gmfd_precip_quantiles_nex-cil-deepsd.nc')
ds_q_gmfd['lon'] = np.where(ds_q_gmfd['lon'] > 180, ds_q_gmfd['lon'] - 360, ds_q_gmfd['lon'])
ds_q_gmfd = ds_q_gmfd.sortby('lon')
    
# Loop through models: RUNTIME IS AROUND 15 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model,
                    [ssp for ssp in list(deepsdbc_dict[model].keys()) if 'pr' in deepsdbc_dict[model][ssp]])
    ds = convert_units(ds)
    
    if ds['pr'].attrs['units'] == 'NaN':
        print(model + ' contains no precip')
        continue
    
    # Select only precip
    ds = ds.drop(['tasmin', 'tasmax'])
    
    ## Calculate metrics
    var_id = 'pr'
    ds_tmp_out = []
    for rp in ['q99', 'rp10']:
        # Get above/below binary
        ds_tmp_q_era5 = ds[var_id] > ds_q_era5[var_id + '_' + rp]
        ds_tmp_q_gmfd = ds[var_id] > ds_q_gmfd[var_id + '_' + rp]
        
        # Count of hot days
        ds_tmp_q_era5_count = ds_tmp_q_era5.resample(time='1Y').sum()
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_count': ds_tmp_q_era5_count}))
        ds_tmp_q_gmfd_count = ds_tmp_q_gmfd.resample(time='1Y').sum()
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_count': ds_tmp_q_gmfd_count}))
        
        # Longest consecutive hot day streak
        ds_tmp_q_era5_streak = ds_tmp_q_era5.resample(time='1Y').apply(n_longest_consecutive)
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_streak': ds_tmp_q_era5_streak}))
        ds_tmp_q_gmfd_streak = ds_tmp_q_gmfd.resample(time='1Y').apply(n_longest_consecutive)
        ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_streak': ds_tmp_q_gmfd_streak}))
        
    # Merge metrics and append
    ds_final = xr.merge(ds_tmp_out)
        
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':20, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/wet/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')
    print(model)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


CanESM5

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'


MRI-ESM2-0
CPU times: user 1min 53s, sys: 921 ms, total: 1min 54s
Wall time: 19min 51s


### Hot days

In [ ]:
%%time
    
# Load quantiles
ds_q_era5 = xr.open_dataset('../data/quantiles/era5_temperature_quantiles_nex-cil-deepsd.nc')
ds_q_era5['lon'] = np.where(ds_q_era5['lon'] > 180, ds_q_era5['lon'] - 360, ds_q_era5['lon'])
ds_q_era5 = ds_q_era5.sortby('lon')

ds_q_gmfd = xr.open_dataset('../data/quantiles/gmfd_temperature_quantiles_nex-cil-deepsd.nc')
ds_q_gmfd['lon'] = np.where(ds_q_gmfd['lon'] > 180, ds_q_gmfd['lon'] - 360, ds_q_gmfd['lon'])
ds_q_gmfd = ds_q_gmfd.sortby('lon')
    
# Loop through models: RUNTIME IS AROUND 10 MINS PER MODEL WITH 40 DASK WORKERS
for model in models:
    # Grab model
    ds = grab_model(method, model, list(deepsdbc_dict[model].keys()))
    ds = convert_units(ds)
    ds['tas'] = (ds['tasmax'] + ds['tasmin']) / 2.
    
    # Drop precip
    ds = ds.drop(['pr'])
    
    ## Calculate metrics
    ds_tmp_final = []
    for var_id in ['tasmin', 'tasmax', 'tas']:
        ds_tmp_out = []
        for rp in ['q99', 'rp10']:
            # Get above/below binary
            ds_tmp_q_era5 = ds[var_id] > ds_q_era5[var_id + '_' + rp]
            ds_tmp_q_gmfd = ds[var_id] > ds_q_gmfd[var_id + '_' + rp]
        
            # Count of hot days
            ds_tmp_q_era5_count = ds_tmp_q_era5.resample(time='1Y').sum()
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_count': ds_tmp_q_era5_count}))
            ds_tmp_q_gmfd_count = ds_tmp_q_gmfd.resample(time='1Y').sum()
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_count': ds_tmp_q_gmfd_count}))
        
            # Longest consecutive hot day streak
            ds_tmp_q_era5_streak = ds_tmp_q_era5.resample(time='1Y').apply(n_longest_consecutive)
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'era5_streak': ds_tmp_q_era5_streak}))
            ds_tmp_q_gmfd_streak = ds_tmp_q_gmfd.resample(time='1Y').apply(n_longest_consecutive)
            ds_tmp_out.append(xr.Dataset({var_id + '_' + rp + 'gmfd_streak': ds_tmp_q_gmfd_streak}))
            
        # Merge RPs and append
        ds_out = xr.merge(ds_tmp_out)
        ds_tmp_final.append(ds_out)
    
    # Merge variables
    ds_final = xr.merge(ds_tmp_final)
    
    # storage options    
    ds_final = ds_final.chunk({'ssp':1, 'time':20, 'lat':720, 'lon':1440})
    
    compressor = zarr.Blosc(cname='zstd', clevel=3)
    encoding = {vname: {'compressor': compressor} for vname in ds_final.data_vars}
    
    azure_prefix = 'carbonplan/' + method + '/hot/' + model
    store = zarr.ABSStore(client=container_client, prefix=azure_prefix)

    # store
    ds_final.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')
    print(model)